In [2]:
from pathlib import Path

import azureml
from IPython.display import display, Markdown
from azureml.core import Run, Model
from azureml.core import Datastore, Experiment, ScriptRunConfig, Workspace, RunConfiguration
from azureml.core.dataset import Dataset
from azureml.data import OutputFileDatasetConfig
from azureml.core.environment import Environment
from azureml.core.runconfig import DockerConfiguration
from azureml.exceptions import UserErrorException

from model_drift import settings
# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (cloudpickle 2.0.0 (d:\code\mlopsday2\medimaging-modeldriftmonitoring\.venv\lib\site-packages), Requirement.parse('cloudpickle<2.0.0,>=1.1.0'), {'azureml-dataprep'}).


Azure ML SDK Version:  1.34.0


In [3]:
# Connect to workspace
ws = Workspace.from_config(settings.AZUREML_CONFIG)

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [27]:
import re

dbg = True
log_refresh_rate = 1

# Name experiement
input_dataset_name="padchest"
experiment_name = 'classifier-padchest'
model_name =  "classifier/padchest-trained/goofy_evening_4p2dpkyb-023.ckpt"
datastore_name = "results"
env_name = "finetune-padchest"
compute_target = "nc6-uswest2"
compute_target = "NC24rs-v3-usw2-d"

#Experiment
exp = Experiment(workspace=ws, name=experiment_name)

#Environment
environment_file = settings.CONDA_ENVIRONMENT_FILE
project_dir = settings.SRC_DIR
pytorch_env = Environment.from_conda_specification(env_name, file_path =str(environment_file))
pytorch_env.register(workspace=ws)
build = pytorch_env.build(workspace=ws)
pytorch_env.environment_variables["RSLEX_DIRECT_VOLUME_MOUNT"] = "True"

# Run Configuration
run_config = RunConfiguration()
run_config.environment_variables["RSLEX_DIRECT_VOLUME_MOUNT"] = "True"
run_config.environment = pytorch_env
run_config.docker = DockerConfiguration(use_docker=True, shm_size="100G")

# Input Dataset
pc_dataset = Dataset.get_by_name(ws, name=input_dataset_name)
datastore = ws.datastores[datastore_name]

print("Experiment:", exp.name)
print("Environment:", pytorch_env.name)

args = [
    "--run_azure", 1,
    "--model", model_name,
    '--data_folder', pc_dataset.as_named_input('padchestv1').as_mount(),
    '--progress_bar_refresh_rate', log_refresh_rate,
    "--log_every_n_steps", log_refresh_rate,
    "--flush_logs_every_n_steps", log_refresh_rate,
    "--batch_size", 32,
    "--accelerator", "ddp",
    "--output_dir", "./outputs/",
    "--append_run_name", 0,
    ]

if dbg:
    args += [
        '--limit_predict_batches', 10,
    ]


config = ScriptRunConfig(
    source_directory = str(project_dir), 
    script = "scripts/finetune/score.py",
    arguments=args,
)
run_config.target = compute_target
config.run_config = run_config

run = exp.submit(config)
display(Markdown(f"""
- Experiment: [{run.experiment.name}]({run.experiment.get_portal_url()})
- Run: [{run.display_name}]({run.get_portal_url()})
- Target: {config.run_config.target}
"""))

Experiment: classifier-padchest
Environment: finetune-padchest



- Experiment: [classifier-padchest](https://ml.azure.com/experiments/classifier-padchest?wsid=/subscriptions/9ca8df1a-bf40-49c6-a13f-66b72a85f43c/resourcegroups/MLOps-Prototype/workspaces/MLOps_shared&tid=72f988bf-86f1-41af-91ab-2d7cd011db47)
- Run: [joyful_pillow_9x1xwn50](https://ml.azure.com/runs/classifier-padchest_1635206413_db9435ce?wsid=/subscriptions/9ca8df1a-bf40-49c6-a13f-66b72a85f43c/resourcegroups/MLOps-Prototype/workspaces/MLOps_shared&tid=72f988bf-86f1-41af-91ab-2d7cd011db47)
- Target: NC24rs-v3-usw2-d
